In [17]:
# hide
# default_exp clean

# Clean Notebooks

In [18]:
# export

from fastcore.script import Param, bool_arg, call_parse
from nbdev.clean import rm_execution_count

In [19]:
# export


def clean_cell(cell, clear_all=False, clear_output=True):
    cell_metadata_keep = nbdev.clean.cell_metadata_keep + ["tags"]
    "Clean `cell` by removing superfluous metadata or everything except the input if `clear_all`"
    rm_execution_count(cell)
    if "outputs" in cell:
        if clear_all or clear_output:
            cell["outputs"] = []
        else:
            clean_cell_output(cell)
    if cell["source"] == [""]:
        cell["source"] = []
    cell["metadata"] = (
        {}
        if clear_all
        else {k: v for k, v in cell["metadata"].items() if k in cell_metadata_keep}
    )

In [20]:
# export


def sciflow_clean_nbs(
    fname: Param("A notebook name or glob to convert", str) = None,
    clear_all: Param("Clean all metadata and outputs", bool_arg) = False,
    disp: Param("Print the cleaned outputs", bool_arg) = False,
    read_input_stream: Param("Read input stram and not nb folder") = False,
):
    "Clean all notebooks in `fname` to avoid merge conflicts"
    # Git hooks will pass the notebooks in the stdin
    if read_input_stream and sys.stdin:
        input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding="utf-8")
        nb = json.load(input_stream)
        clean_nb(nb, clear_all=clear_all)
        _print_output(nb)
        return
    path = None
    if fname is None:
        try:
            path = get_config().path("nbs_path")
        except Exception:
            path = Path.cwd()

    files = nbglob(fname=ifnone(fname, path))
    for f in files:
        if not str(f).endswith(".ipynb"):
            continue
        nb = json.loads(open(f, "r", encoding="utf-8").read())
        clean_nb(nb, clear_all=clear_all)
        if disp:
            _print_output(nb)
        else:
            x = json.dumps(nb, sort_keys=True, indent=1, ensure_ascii=False)
            with io.open(f, "w", encoding="utf-8") as f:
                f.write(x)
                f.write("\n")

In [21]:
# export


@call_parse
def sciflow_clean():
    sciflow_clean_nbs()